<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/research/modeling_PolyEncoder/test_sample_PolyEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q transformers
! pip install -q datasets

In [ ]:
! rm -rf PyTorch-Architectures/
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer
from research import PolyEncoder

In [ ]:
dataset = load_dataset("glue", "mrpc")

In [13]:
def print_details(samples):
  labels = []
  max_len_sent1, max_len_sent2 = -1, -1
  min_len_sent1, min_len_sent2 = 100, 100
  for sample in samples:
    max_len_sent1 = max(len(sample["sentence1"].split()), max_len_sent1)
    max_len_sent2 = max(len(sample["sentence2"].split()), max_len_sent2)

    min_len_sent1 = min(len(sample["sentence1"].split()), min_len_sent1)
    min_len_sent2 = min(len(sample["sentence2"].split()), min_len_sent2)
    labels.append(sample["label"])
  
  print(f"Total pairs --> {len(samples)}")
  print(f"No of positive pairs --> {sum(labels)}")
  print(f"No of negative pairs --> {len(labels) - sum(labels)}")

  print(f"Maximum length of sentence1 --> {max_len_sent1}")
  print(f"Maximum length of sentence2 --> {max_len_sent2}")

  print(f"Minimum length of sentence1 --> {min_len_sent1}")
  print(f"Minimum length of sentence2 --> {min_len_sent2}")

print(f"For Training Set:")
print_details(dataset["train"])

print(f"\nFor Validation Set:")
print_details(dataset["validation"])

print(f"\nFor Test Set:")
print_details(dataset["test"])

For Training Set:
Total pairs --> 3668
No of positive pairs --> 2474
No of negative pairs --> 1194
Maximum length of sentence1 --> 39
Maximum length of sentence2 --> 42
Minimum length of sentence1 --> 7
Minimum length of sentence2 --> 8

For Validation Set:
Total pairs --> 408
No of positive pairs --> 279
No of negative pairs --> 129
Maximum length of sentence1 --> 35
Maximum length of sentence2 --> 34
Minimum length of sentence1 --> 9
Minimum length of sentence2 --> 9

For Test Set:
Total pairs --> 1725
No of positive pairs --> 1147
No of negative pairs --> 578
Maximum length of sentence1 --> 36
Maximum length of sentence2 --> 35
Minimum length of sentence1 --> 8
Minimum length of sentence2 --> 7
